# Spider Fly testing environment

In [1]:
from custom_agent.CTCE.ma_sac_agents_seq_discrete import Agents
from custom_spider_env.spider_fly_env.envs.grid_MA_testing import SpiderFlyEnvMA
from custom_spider_env.spider_fly_env.envs.pettingzoo_wrapper import PettingZooWrapper

import numpy as np

import pandas as pd

env = SpiderFlyEnvMA(max_steps = 100)
env = PettingZooWrapper(env)

agents = Agents(env, batch_size = 256, layer_sizes = (64, 64), global_observations = True) 
# agents = Agents(env, batch_size = 256, global_observations = True) 
rewards = agents.train(nr_steps = 30000)

/home/hkolstee/.local/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment SpiderFlyGrid-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/hkolstee/.local/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment SpiderFlyGridMA-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/hkolstee/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return 

[Episode 50 total reward: [20. 16.]] ~ 
[Episode 100 total reward: [21. 15.]] ~ 
[Episode 150 total reward: [5. 6.]] ~ 


KeyboardInterrupt: 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

data = np.vstack(rewards)

df = pd.DataFrame(data, columns = ["agent_" + str(idx) for idx in range(agents.nr_agents)])
df["Episode"] = list(range(data.shape[0]))

df = df.melt('Episode', var_name='Agent', value_name='Rewards')

df["Avg_Reward"] = df["Rewards"].rolling(window = 50, step = 50).mean()
df = df[df.Episode > 50]
df.dropna()


sns.lineplot(data = df, x = "Episode", y = "Avg_Reward")
plt.figure()
sns.lineplot(data = df, x = "Episode", y = "Avg_Reward", hue = "Agent")

# TEST of custom Multi-Agent SAC agent on citylearn env

In [ ]:
import gymnasium as gym
import seaborn as sns
import matplotlib.pyplot as plt

import torch

import numpy as np
import pandas as pd

from citylearn.citylearn import CityLearnEnv
from citylearn.wrappers import NormalizedSpaceWrapper, StableBaselines3Wrapper

from custom_agent.CTDE.ma_sac_agents_seq2 import Agents

from custom_reward.custom_reward import CustomReward

In [ ]:
schema_path = "data/schema.json"

env = CityLearnEnv(schema = schema_path, reward_function = CustomReward, central_agent=False)

# wrap environment for a more workable env
env = NormalizedSpaceWrapper(env)

In [ ]:
sac_agent = Agents(env, batch_size=100, buffer_max_size=100000)

In [ ]:
env.observation_space

In [ ]:
env.action_space

In [ ]:
warmup_steps = 5000

# make agent
sac_agent = Agents(env, batch_size=256, buffer_max_size=100000)

# training run
sac_agent.train(nr_steps = 5500 * 720, warmup_steps = warmup_steps, learn_delay = 100, learn_freq = 1, learn_weight = 1, save_dir = "temp_models")
